In [1]:
# import necessary package
import tensorflow as tf
import numpy as np
import pathlib
import datetime

# printout versions
print(f"Tensor Flow Version: {tf.__version__}")
print(f"numpy Version: {np.version.version}")

Tensor Flow Version: 2.12.0
numpy Version: 1.23.5


In [2]:
# Raw Dataset Directory
data_dir = pathlib.Path("./alexnetnew/80% of training image/")
image_count = len(list(data_dir.glob('*/*.jpg')))
print("Total Training number of images:", image_count)

data_dir = pathlib.Path("./alexnetnew/20% of testing image/")
image_count = len(list(data_dir.glob('*/*.jpg')))
print("Total Testing number of images:", image_count)

# class names in the dataset specified
CLASS_NAMES = np.array([item.name for item in data_dir.glob('*') if item.name != "LICENSE.txt"])
print("Class names:", CLASS_NAMES)

# print length of class names
output_class_units = len(CLASS_NAMES)
print("Number of output units:", output_class_units)

Total Training number of images: 4800
Total Testing number of images: 1200
Class names: ['new glioma after deleting' 'new meningioma after deleting'
 'new notumor after deleting' 'new pituitary after deleting']
Number of output units: 4


In [3]:

model = tf.keras.models.Sequential([
    # 1st Convolutional Layer
  tf.keras.layers.Conv2D(96, (11,11),strides=(4,4), activation='relu', input_shape=(227, 227, 3)),
  tf.keras.layers.BatchNormalization(),
  tf.keras.layers.MaxPooling2D(2, strides=(2,2)),
    # 2nd conv
  tf.keras.layers.Conv2D(256, (11,11),strides=(1,1), activation='relu',padding="same"),
  tf.keras.layers.BatchNormalization(),
     # 3rd conv
  tf.keras.layers.Conv2D(384, (3,3),strides=(1,1), activation='relu',padding="same"),
  tf.keras.layers.BatchNormalization(),
    # 4th conv
  tf.keras.layers.Conv2D(384, (3,3),strides=(1,1), activation='relu',padding="same"),
  tf.keras.layers.BatchNormalization(),
    # 5th Conv
  tf.keras.layers.Conv2D(256, (3, 3), strides=(1, 1), activation='relu',padding="same"),
  tf.keras.layers.BatchNormalization(),
  tf.keras.layers.MaxPooling2D(2, strides=(2, 2)),
  # To Flatten layer
  tf.keras.layers.Flatten(),
  # Fully connected Layer 1
  tf.keras.layers.Dense(4096, activation='relu'),
    # add dropout 0.5 ==> tf.keras.layers.Dropout(0.5),
  #To FC layer 2
  tf.keras.layers.Dense(4096, activation='relu'),
    
    # Output Layer
    # add dropout 0.5 ==> tf.keras.layers.Dropout(0.5),
  tf.keras.layers.Dense(output_class_units, activation='softmax')
])

In [4]:
# Define input shape parameters
BATCH_SIZE = 32       # Batch size for training
IMG_HEIGHT = 227       # Height of the input images
IMG_WIDTH = 227        # Width of the input images

# Calculate steps per epoch
STEPS_PER_EPOCH = np.ceil(image_count / BATCH_SIZE)

# Preprocess the images
image_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

# Generate training data batches
train_data_gen = image_generator.flow_from_directory(
    directory=str(data_dir),
    batch_size=BATCH_SIZE,
    shuffle=True,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    classes=list(CLASS_NAMES)
)

Found 1200 images belonging to 4 classes.


In [5]:
# # Specifying the optimizer, Loss function for optimization & Metrics to be displayed
# model.compile(optimizer='sgd', loss="categorical_crossentropy", metrics=['accuracy'])

# # Summarizing the model architecture and printing it out
# model.summary()


# Added Learning Rate and sgd optimizer

# Compile the model with SGD optimizer and learning rate
learning_rate = 0.0001  # (change) Learning rate for the SGD optimizer
sgd_optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate, momentum=0.9)
model.compile(optimizer=sgd_optimizer, loss="categorical_crossentropy", metrics=['accuracy'])

# Print model summary
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 55, 55, 96)        34944     
                                                                 
 batch_normalization (BatchN  (None, 55, 55, 96)       384       
 ormalization)                                                   
                                                                 
 max_pooling2d (MaxPooling2D  (None, 27, 27, 96)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 27, 27, 256)       2973952   
                                                                 
 batch_normalization_1 (Batc  (None, 27, 27, 256)      1024      
 hNormalization)                                                 
                                                        

In [6]:
# Define custom callback to stop training
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if logs.get("accuracy") == 1.0 and logs.get("loss") < 0.03:
            print("\nReached 100% accuracy so stopping training")
            self.model.stop_training = True

callbacks = myCallback()

# Setup TensorBoard callback
log_dir = "logs\\fit\\" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=0)

In [7]:
# Train the model
history = model.fit(
    train_data_gen,
    steps_per_epoch=STEPS_PER_EPOCH,
    epochs=10,
    callbacks=[tensorboard_callback, callbacks]
)

# Save the entire model
model.save('AlexNet_saved_model.h5')

Epoch 1/10
38/38 [==============================] - 498s 13s/step - loss: 0.8583 - accuracy: 0.6500
Epoch 2/10
38/38 [==============================] - 491s 13s/step - loss: 0.3104 - accuracy: 0.8883
Epoch 3/10
38/38 [==============================] - 499s 13s/step - loss: 0.1363 - accuracy: 0.9667
Epoch 4/10
38/38 [==============================] - 491s 13s/step - loss: 0.0582 - accuracy: 0.9917
Epoch 5/10
38/38 [==============================] - 496s 13s/step - loss: 0.0405 - accuracy: 1.0000
Epoch 6/10
38/38 [==============================] - ETA: 0s - loss: 0.0212 - accuracy: 1.0000 
Reached 100% accuracy so stopping training
38/38 [==============================] - 490s 13s/step - loss: 0.0212 - accuracy: 1.0000
